<a href="https://colab.research.google.com/github/talhaanwarch/mia-covid19/blob/main/ag2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install packages

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f03c6139-eaa3-9628-670d-637a91e33bcf)


In [2]:
#restart kernel after installing packages and run it again
%%capture 
!pip install -U  pip
!pip install -U setuptools wheel
!pip install -U "mxnet_cu110<2.0.0"
!pip install  autogluon  # autogluon==0.2.0

# Get data from drive

In [3]:
#connect drive with colab notebook/
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [5]:
#copy data from drive to colab
!cp /content/drive/MyDrive/covid/train.zip /content/
!cp /content/drive/MyDrive/covid/val.zip /content/

In [6]:
#unzip data
%%capture 
!unzip /content/train.zip
!rm /content/train.zip
!mv /content/resized /content/train
#val data
!unzip /content/val.zip -d val
!rm /content/val.zip


# import packages

In [4]:
import pandas as pd
import numpy as np
from glob import glob
import cv2
import pandas as pd
import autogluon.core as ag
from autogluon.vision import ImagePredictor
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


# prepare data

In [5]:
glob('/content/train'+'/*/')[1].split('/')[-2]

'non-covid'

In [7]:
def flatten(path,label=None):
  '''
  path: images path in a list
  label: corresponding label in list
  '''
  path = [item for sublist in path for item in sublist]
  label = [item for sublist in label for item in sublist]
  return path, label
  

def prepare(path,split=False,flat=True):
  '''
  Pass the path of main folder in which binary class folders are present
  Split True will split the data and return two sets
  '''
  main_path=glob(path+'/*/')
  pos_path=[glob(folder+'*.jpg') for folder in glob(main_path[0]+'*/')]
  neg_path=[glob(folder+'*.jpg') for folder in glob(main_path[1]+'*/')]
  #get folder name as label
  pos_label=[[main_path[0].split('/')[-2]]*len(i)  for i in pos_path]
  neg_label=[[main_path[1].split('/')[-2]]*len(i)  for i in neg_path]
  path=pos_path+neg_path
  label=pos_label+neg_label
  if flat:
    if split:
      train_path,val_path,train_label,val_label=train_test_split(path,label,test_size=0.2)
      train_path,train_label=flatten(train_path,train_label)
      val_path,val_label=flatten(val_path,val_label)
      return train_path,val_path,train_label,val_label
    else:
      path,label=flatten(path,label)
      return path,label
  else:
    return path,label
  

In [8]:
# train_path,val_path,train_label,val_label=prepare('/content/train',split=True)
# test_path,test_label=prepare('/content/val',split=False)
# print('Train Images and labels',len(train_path),len(train_label))
# print('Val Images and labels',len(val_path),len(val_label))
# print('Test Images and labels',len(test_path),len(test_label))

In [9]:
# train_df=pd.DataFrame(zip(train_path,train_label),columns=['image','label'])
# train_df = train_df.sample(frac=1).reset_index(drop=True)
# train_df.label=train_df.label.map({'non-covid':0,'covid':1})
# train_df.to_csv('/content/drive/MyDrive/covid/train_df.csv',index=False)
# train_df.head()

In [10]:
# val_df=pd.DataFrame(zip(val_path,val_label),columns=['image','label'])
# val_df = val_df.sample(frac=1).reset_index(drop=True)
# val_df.label=val_df.label.map({'non-covid':0,'covid':1})
# val_df.to_csv('/content/drive/MyDrive/covid/val_df.csv',index=False)
# val_df.head()

In [11]:
# test_df=pd.DataFrame(zip(test_path,test_label),columns=['image','label'])
# test_df = test_df.sample(frac=1).reset_index(drop=True)
# test_df.label=test_df.label.map({'non-covid':0,'covid':1})
# test_df.to_csv('/content/drive/MyDrive/covid/test_df.csv',index=False)
# test_df.head()

In [12]:
train_df=pd.read_csv('/content/drive/MyDrive/covid/train_df.csv')

val_df=pd.read_csv('/content/drive/MyDrive/covid/val_df.csv')

test_df=pd.read_csv('/content/drive/MyDrive/covid/test_df.csv')


# train model

In [13]:
%%time
predictor = ImagePredictor(verbosity=1)
predictor.fit(train_df,tuning_data=val_df) 

The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1


CPU times: user 2h 17min 54s, sys: 3min 36s, total: 2h 21min 30s
Wall time: 2h 4min 15s


In [14]:
fit_result = predictor.fit_summary()
print('Top-1 train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))
fit_result


Top-1 train acc: 0.839, val acc: 0.767


{'best_config': {'batch_size': 16,
  'dist_ip_addrs': None,
  'early_stop_baseline': -inf,
  'early_stop_max_value': inf,
  'early_stop_patience': 10,
  'epochs': 15,
  'estimator': gluoncv.auto.estimators.image_classification.image_classification.ImageClassificationEstimator,
  'final_fit': False,
  'gpus': [0],
  'log_dir': '/content/3251e714',
  'lr': 0.01,
  'model': 'resnet50_v1b',
  'ngpus_per_trial': 8,
  'nthreads_per_trial': 128,
  'num_trials': 1,
  'num_workers': 2,
  'scheduler': 'local',
  'search_strategy': 'random',
  'searcher': 'random',
  'seed': 320,
  'time_limits': 7200,
  'wall_clock_tick': 1621737706.1998365},
 'fit_history': {'best_config': {'batch_size': 16,
   'dist_ip_addrs': None,
   'early_stop_baseline': -inf,
   'early_stop_max_value': inf,
   'early_stop_patience': 10,
   'epochs': 15,
   'estimator': gluoncv.auto.estimators.image_classification.image_classification.ImageClassificationEstimator,
   'final_fit': False,
   'gpus': [0],
   'log_dir': '/cont

In [15]:
predictor.save('/content/drive/MyDrive/covid/predictor_mqft.ag')


In [16]:
val_df=ImagePredictor.Dataset.from_csv('/content/drive/MyDrive/covid/val_df.csv')
res = predictor.evaluate(val_df)
print('Top-1 test acc: %.3f' % res[0])
val_pred=predictor.predict(test_df)
print(classification_report(test_df.label,val_pred))

Top-1 test acc: 0.767
              precision    recall  f1-score   support

           0       0.75      0.92      0.83     40516
           1       0.88      0.65      0.75     35016

    accuracy                           0.80     75532
   macro avg       0.81      0.79      0.79     75532
weighted avg       0.81      0.80      0.79     75532



In [17]:
test_df=ImagePredictor.Dataset.from_csv('/content/drive/MyDrive/covid/test_df.csv')
res = predictor.evaluate(test_df)
print('Top-1 test acc: %.3f' % res[0])

Top-1 test acc: 0.796


In [18]:
test_pred=predictor.predict(test_df)
print(classification_report(test_df.label,test_pred))

              precision    recall  f1-score   support

           0       0.75      0.92      0.83     40516
           1       0.88      0.65      0.75     35016

    accuracy                           0.80     75532
   macro avg       0.81      0.79      0.79     75532
weighted avg       0.81      0.80      0.79     75532



# final evaluation on validation data

In [19]:
#predictor=predictor.load('/content/drive/MyDrive/covid/predictor.ag')


In [20]:
path,label=prepare('/content/val',split=False,flat=False)
len(path),len(label)

(374, 374)

In [21]:
#now loop each folder of each directoy and create a dataframe for each folder
def evaluate(path,label):
  test_pred,test_true=[],[]
  for i, j in zip(path,label):
    test_df=pd.DataFrame(zip(i,j),columns=['image','label'])
    test_df.label=test_df.label.map({'non-covid':0,'covid':1})
    test_df.to_csv('test_df.csv',index=False)
    test_df=ImagePredictor.Dataset.from_csv('/content/test_df.csv')
    pred=list(predictor.predict(test_df))
    #find maximum occurent of element
    test_pred.append(max(pred,key=pred.count))
    test_true.append(test_df.label.mean())
  return test_pred,test_true



In [22]:
test_pred,test_true=evaluate(path,label)

In [23]:
print(classification_report(test_true,test_pred,target_names=['non-covid','covid']))

              precision    recall  f1-score   support

   non-covid       0.79      0.99      0.88       209
       covid       0.98      0.67      0.79       165

    accuracy                           0.85       374
   macro avg       0.89      0.83      0.84       374
weighted avg       0.87      0.85      0.84       374

